//Chat GPT Iteration\\

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont
import emoji

class ElementGame:
    import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont

class ElementGame:
    def __init__(self, cap, width, height):
        self.cap = cap
        self.width = width
        self.height = height
        self.square_size = min(width, height) // 4
        
        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
        
        # Initialize drawing utilities
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.squares = {
            '🔥': {'color': (255, 0, 0), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},  # Red for fire
            '💨': {'color': (173, 216, 230), 'position': (width - self.square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},  # Light blue for air
            '🌊': {'color': (0, 0, 139), 'position': (0, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},  # Dark blue for water
            '🌱': {'color': (139, 69, 19), 'position': (width - self.square_size, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}  # Brown for earth
        }
        
        self.gold_box = {'color': (0, 215, 255), 'position': ((width - self.square_size) // 2, height - self.square_size)}
        self.eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'
        
        self.sound_played = {element: False for element in self.squares}
        self.finger_in_box = {element: False for element in self.squares}
        self.last_sound_time = {element: 0 for element in self.squares}
        
        self.grabbed_word = None
        self.gold_achieved = False
        self.mask_color = None
        
        self.reset_word_positions()
        
        # Use the Segoe UI Emoji font for rendering emojis
        self.emoji_font = ImageFont.truetype("seguiemj.ttf", 48)
        self.emoji_images = self.prepare_emoji_images()

    def reset_word_positions(self):
        self.word_positions = {element: (info['position'][0] + self.square_size // 2, info['position'][1] + self.square_size // 2) 
                               for element, info in self.squares.items()}
        self.gold_achieved = False
        self.grabbed_word = None
        self.mask_color = None

    def is_hand_closed(self, hand_landmarks):
        thumb_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.THUMB_TIP]
        index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
        return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

    def is_point_in_box(self, point, box_position, box_size):
        return (box_position[0] < point[0] < box_position[0] + box_size and
                box_position[1] < point[1] < box_position[1] + box_size)

    def prepare_emoji_images(self):
        emoji_images = {}
        for element in self.squares.keys():
            size = self.square_size
            emoji_images[element] = Image.new('RGBA', (size, size), (255, 255, 255, 0))
            d = ImageDraw.Draw(emoji_images[element])
            text_bbox = d.textbbox((0, 0), element, font=self.emoji_font)
            text_x = (size - (text_bbox[2] - text_bbox[0])) // 2
            text_y = (size - (text_bbox[3] - text_bbox[1])) // 2
            d.text((text_x, text_y), element, font=self.emoji_font, fill=(255, 255, 255, 255))
        return emoji_images

    def play_sound(self, sound_file):
        def play():
            try:
                winsound.PlaySound(sound_file, winsound.SND_FILENAME)
                print(f"Played sound: {sound_file}")
            except Exception as e:
                print(f"Error playing sound {sound_file}: {e}")

        threading.Thread(target=play, daemon=True).start()

    def combine_colors(self, colors):
        """Combine multiple RGB colors safely."""
        combined = np.clip(np.mean(colors, axis=0), 0, 255).astype(int)
        return tuple(combined)

    def validate_color(self, color):
        """Ensure the color is a valid RGB tuple."""
        if isinstance(color, tuple) and len(color) == 3:
            return tuple(int(c) for c in color)
        else:
            return (255, 255, 255)  # Fallback to white if invalid

    def process_frame(self, frame):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)

        overlay = image_rgb.copy()
        for element, info in self.squares.items():
            color = self.validate_color(info['color'])
            cv2.rectangle(overlay, info['position'], 
                          (info['position'][0] + self.square_size, info['position'][1] + self.square_size), 
                          color, -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        color = self.validate_color(self.gold_box['color'])
        cv2.rectangle(overlay, self.gold_box['position'], 
                      (self.gold_box['position'][0] + self.square_size, self.gold_box['position'][1] + self.square_size), 
                      color, -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        pil_image = Image.fromarray(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    image_rgb, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                    self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
                )
                
                index_finger_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * self.width), int(index_finger_tip.y * self.height)

                hand_closed = self.is_hand_closed(hand_landmarks)

                if self.grabbed_word:
                    if hand_closed:
                        self.word_positions[self.grabbed_word] = (x, y)
                    else:
                        if self.is_point_in_box((x, y), self.gold_box['position'], self.square_size):
                            self.mask_color = self.squares[self.grabbed_word]['color']
                            self.gold_box['color'] = self.mask_color
                        self.grabbed_word = None
                else:
                    for element, info in self.squares.items():
                        if self.is_point_in_box((x, y), info['position'], self.square_size):
                            current_time = time.time()
                            if not self.finger_in_box[element]:
                                print(f"Touch detected in {element} square")
                                self.finger_in_box[element] = True
                                if current_time - self.last_sound_time[element] > 1:  # 1 second cooldown
                                    print(f"Attempting to play sound for {element}")
                                    self.play_sound(info['sound'])
                                    self.last_sound_time[element] = current_time
                            if hand_closed and not self.grabbed_word:
                                self.grabbed_word = element
                        else:
                            self.finger_in_box[element] = False

        # Check if multiple elements are in the gold box
        colors_in_gold = [self.squares[element]['color'] for element, position in self.word_positions.items() if self.is_point_in_box(position, self.gold_box['position'], self.square_size)]
        if len(colors_in_gold) > 1:
            combined_color = self.combine_colors(colors_in_gold)
            self.mask_color = self.validate_color(combined_color)
            self.gold_box['color'] = self.mask_color
            print(f"Combined color applied: {self.mask_color}")
        elif len(colors_in_gold) == 1:
            self.mask_color = self.validate_color(colors_in_gold[0])
            self.gold_box['color'] = self.mask_color

        for element, position in self.word_positions.items():
            pil_image.paste(self.emoji_images[element], (position[0] - self.square_size // 2, position[1] - self.square_size // 2), self.emoji_images[element])

        return np.array(pil_image)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    image_rgb, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                    self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
                )
                
                index_finger_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * self.width), int(index_finger_tip.y * self.height)

                hand_closed = self.is_hand_closed(hand_landmarks)

                if self.grabbed_word:
                    if hand_closed:
                        self.word_positions[self.grabbed_word] = (x, y)
                    else:
                        if self.is_point_in_box((x, y), self.gold_box['position'], self.square_size):
                            self.mask_color = self.squares[self.grabbed_word]['color']
                            self.gold_box['color'] = self.mask_color
                        self.grabbed_word = None
                else:
                    for element, info in self.squares.items():
                        if self.is_point_in_box((x, y), info['position'], self.square_size):
                            current_time = time.time()
                            if not self.finger_in_box[element]:
                                print(f"Touch detected in {element} square")
                                self.finger_in_box[element] = True
                                if current_time - self.last_sound_time[element] > 1:  # 1 second cooldown
                                    print(f"Attempting to play sound for {element}")
                                    self.play_sound(info['sound'])
                                    self.last_sound_time[element] = current_time
                            if hand_closed and not self.grabbed_word:
                                self.grabbed_word = element
                        else:
                            self.finger_in_box[element] = False

        # Check if multiple elements are in the gold box
        colors_in_gold = [self.squares[element]['color'] for element, position in self.word_positions.items() if self.is_point_in_box(position, self.gold_box['position'], self.square_size)]
        if len(colors_in_gold) > 1:
            combined_color = self.combine_colors(colors_in_gold)
            self.mask_color = self.validate_color(combined_color)
            self.gold_box['color'] = self.mask_color
            print(f"Combined color applied: {self.mask_color}")
        elif len(colors_in_gold) == 1:
            self.mask_color = self.validate_color(colors_in_gold[0])
            self.gold_box['color'] = self.mask_color

        for element, position in self.word_positions.items():
            pil_image.paste(self.emoji_images[element], (position[0] - self.square_size // 2, position[1] - self.square_size // 2), self.emoji_images[element])

        return np.array(pil_image)


class CameraManager:
    def __init__(self, cap):
        self.cap = cap
        self.is_game_view = True
        self.mp_selfie_segmentation = mp.solutions.selfie_segmentation
        self.selfie_segmentation = self.mp_selfie_segmentation.SelfieSegmentation(model_selection=0)

    def switch_view(self):
        self.is_game_view = not self.is_game_view
        message = 'Game View' if self.is_game_view else 'Mask View'
        print(f"Switched to {message}")
        # Optionally display a message on the screen

    def apply_mask(self, frame, mask_color):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.selfie_segmentation.process(image_rgb)
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        if mask_color is not None:
            overlay = np.zeros(image_rgb.shape, dtype=np.uint8)
            overlay[:] = mask_color
            overlay = cv2.addWeighted(image_rgb, 0.55, overlay, 0.45, 0)
            image_rgb = np.where(condition, overlay, image_rgb)
        return image_rgb

class MainProgram:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        ret, frame = self.cap.read()
        self.height, self.width = frame.shape[:2]
        self.game = ElementGame(self.cap, self.width, self.height)
        self.camera_manager = CameraManager(self.cap)

    def run(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to capture frame")
                break

            frame = cv2.flip(frame, 1)

            if self.camera_manager.is_game_view:
                image_rgb = self.game.process_frame(frame)
            else:
                image_rgb = self.camera_manager.apply_mask(frame, self.game.mask_color)

            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            cv2.imshow('Camera View', image_bgr)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                self.game.reset_word_positions()
                print("Game reset!")
            elif key == ord('s'):
                self.camera_manager.switch_view()

        self.cap.release()
        cv2.destroyAllWindows()
        winsound.PlaySound(None, winsound.SND_PURGE)

if __name__ == "__main__":
    program = MainProgram()
    program.run()


c:\Users\dsrus\Desktop\Workspace\MTLiens\pytorch_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Touch detected in 🌊 square
Attempting to play sound for 🌊
Switched to Mask View
Switched to Game View
Switched to Mask View
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav
Switched to Game View
Switched to Mask View
Switched to Game View


//refined\\

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont

class ElementGame:
    def __init__(self, cap, width, height):
        self.cap = cap
        self.width = width
        self.height = height
        self.square_size = min(width, height) // 4
        
        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
        
        # Initialize drawing utilities
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.squares = {
            '🔥': {'color': (255, 0, 0), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},  # Red for fire
            '💨': {'color': (173, 216, 230), 'position': (width - self.square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},  # Light blue for air
            '🌊': {'color': (0, 0, 139), 'position': (0, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},  # Dark blue for water
            '🌱': {'color': (139, 69, 19), 'position': (width - self.square_size, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}  # Brown for earth
        }
        
        self.gold_box = {'color': (0, 215, 255), 'position': ((width - self.square_size) // 2, height - self.square_size)}
        self.eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'
        
        self.sound_played = {element: False for element in self.squares}
        self.finger_in_box = {element: False for element in self.squares}
        self.last_sound_time = {element: 0 for element in self.squares}
        
        self.grabbed_word = None
        self.gold_achieved = False
        self.mask_color = None
        
        self.reset_word_positions()
        
        # Use the Segoe UI Emoji font for rendering emojis
        self.emoji_font = ImageFont.truetype("seguiemj.ttf", 48)
        self.emoji_images = self.prepare_emoji_images()

    def reset_word_positions(self):
        self.word_positions = {element: (info['position'][0] + self.square_size // 2, info['position'][1] + self.square_size // 2) 
                               for element, info in self.squares.items()}
        self.gold_achieved = False
        self.grabbed_word = None
        self.mask_color = None

    def is_hand_closed(self, hand_landmarks):
        thumb_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.THUMB_TIP]
        index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
        return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

    def is_point_in_box(self, point, box_position, box_size):
        return (box_position[0] < point[0] < box_position[0] + box_size and
                box_position[1] < point[1] < box_position[1] + box_size)

    def prepare_emoji_images(self):
        emoji_images = {}
        for element in self.squares.keys():
            size = self.square_size
            emoji_images[element] = Image.new('RGBA', (size, size), (255, 255, 255, 0))
            d = ImageDraw.Draw(emoji_images[element])
            text_bbox = d.textbbox((0, 0), element, font=self.emoji_font)
            text_x = (size - (text_bbox[2] - text_bbox[0])) // 2
            text_y = (size - (text_bbox[3] - text_bbox[1])) // 2
            d.text((text_x, text_y), element, font=self.emoji_font, fill=(255, 255, 255, 255))
        return emoji_images

    def play_sound(self, sound_file):
        def play():
            try:
                winsound.PlaySound(sound_file, winsound.SND_FILENAME)
                print(f"Played sound: {sound_file}")
            except Exception as e:
                print(f"Error playing sound {sound_file}: {e}")

        threading.Thread(target=play, daemon=True).start()

    def combine_colors(self, colors):
        """Combine multiple RGB colors safely."""
        combined = np.clip(np.mean(colors, axis=0), 0, 255).astype(int)
        return tuple(combined)

    def validate_color(self, color):
        """Ensure the color is a valid RGB tuple."""
        if isinstance(color, tuple) and len(color) == 3:
            return tuple(int(c) for c in color)
        else:
            return (255, 255, 255)  # Fallback to white if invalid

    def process_frame(self, frame):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)

        overlay = image_rgb.copy()
        for element, info in self.squares.items():
            color = self.validate_color(info['color'])
            cv2.rectangle(overlay, info['position'], 
                          (info['position'][0] + self.square_size, info['position'][1] + self.square_size), 
                          color, -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        color = self.validate_color(self.gold_box['color'])
        cv2.rectangle(overlay, self.gold_box['position'], 
                      (self.gold_box['position'][0] + self.square_size, self.gold_box['position'][1] + self.square_size), 
                      color, -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        pil_image = Image.fromarray(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    image_rgb, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                    self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
                )
                
                index_finger_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * self.width), int(index_finger_tip.y * self.height)

                hand_closed = self.is_hand_closed(hand_landmarks)

                if self.grabbed_word:
                    if hand_closed:
                        self.word_positions[self.grabbed_word] = (x, y)
                    else:
                        if self.is_point_in_box((x, y), self.gold_box['position'], self.square_size):
                            self.mask_color = self.squares[self.grabbed_word]['color']
                            self.gold_box['color'] = self.mask_color
                        self.grabbed_word = None
                else:
                    for element, info in self.squares.items():
                        if self.is_point_in_box((x, y), info['position'], self.square_size):
                            current_time = time.time()
                            if not self.finger_in_box[element]:
                                print(f"Touch detected in {element} square")
                                self.finger_in_box[element] = True
                                if current_time - self.last_sound_time[element] > 1:  # 1 second cooldown
                                    print(f"Attempting to play sound for {element}")
                                    self.play_sound(info['sound'])
                                    self.last_sound_time[element] = current_time
                            if hand_closed and not self.grabbed_word:
                                self.grabbed_word = element
                        else:
                            self.finger_in_box[element] = False

        # Check if multiple elements are in the gold box
        colors_in_gold = [self.squares[element]['color'] for element, position in self.word_positions.items() if self.is_point_in_box(position, self.gold_box['position'], self.square_size)]
        if len(colors_in_gold) > 1:
            combined_color = self.combine_colors(colors_in_gold)
            self.mask_color = self.validate_color(combined_color)
            self.gold_box['color'] = self.mask_color
            print(f"Combined color applied: {self.mask_color}")
        elif len(colors_in_gold) == 1:
            self.mask_color = self.validate_color(colors_in_gold[0])
            self.gold_box['color'] = self.mask_color

        for element, position in self.word_positions.items():
            pil_image.paste(self.emoji_images[element], (position[0] - self.square_size // 2, position[1] - self.square_size // 2), self.emoji_images[element])

        return np.array(pil_image)

class CameraManager:
    def __init__(self, cap):
        self.cap = cap
        self.is_game_view = True
        self.mp_selfie_segmentation = mp.solutions.selfie_segmentation
        self.selfie_segmentation = self.mp_selfie_segmentation.SelfieSegmentation(model_selection=0)

    def switch_view(self):
        self.is_game_view = not self.is_game_view
        message = 'Game View' if self.is_game_view else 'Mask View'
        print(f"Switched to {message}")

    def apply_mask(self, frame, mask_color, opacity):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.selfie_segmentation.process(image_rgb)
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        if mask_color is not None:
            overlay = np.full(image_rgb.shape, mask_color, dtype=np.uint8)
            image_rgb = np.where(condition, cv2.addWeighted(image_rgb, 1 - opacity, overlay, opacity, 0), image_rgb)
        return image_rgb

class MainProgram:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        ret, frame = self.cap.read()
        self.height, self.width = frame.shape[:2]
        self.game = ElementGame(self.cap, self.width, self.height)
        self.camera_manager = CameraManager(self.cap)

    def run(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to capture frame")
                break

            frame = cv2.flip(frame, 1)

            if self.camera_manager.is_game_view:
                image_rgb = self.game.process_frame(frame)
                image_rgb = self.camera_manager.apply_mask(image_rgb, self.game.mask_color, opacity=0.25)
            else:
                image_rgb = self.camera_manager.apply_mask(frame, self.game.mask_color, opacity=0.45)

            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            cv2.imshow('Camera View', image_bgr)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                self.game.reset_word_positions()
                print("Game reset!")
            elif key == ord('s'):
                self.camera_manager.switch_view()

        self.cap.release()
        cv2.destroyAllWindows()
        winsound.PlaySound(None, winsound.SND_PURGE)

if __name__ == "__main__":
    program = MainProgram()
    program.run()

Switched to Mask View
Switched to Game View
Touch detected in 🌊 square
Attempting to play sound for 🌊
Switched to Mask View
Switched to Game View
Touch detected in 🌱 square
Attempting to play sound for 🌱
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Combined color applied: (69, 34, 79)
Switched to Mask View
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav


In [6]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont

class ElementGame:
    def __init__(self, cap, width, height):
        self.cap = cap
        self.width = width
        self.height = height
        self.square_size = min(width, height) // 4
        
        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
        
        # Initialize drawing utilities
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.squares = {
            '🔥': {'color': (255, 0, 0), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},  # Red for fire
            '💨': {'color': (173, 216, 230), 'position': (width - self.square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},  # Light blue for air
            '🌊': {'color': (0, 0, 139), 'position': (0, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},  # Dark blue for water
            '🌱': {'color': (139, 69, 19), 'position': (width - self.square_size, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}  # Brown for earth
        }
        
        self.gold_box = {'color': (0, 215, 255), 'position': ((width - self.square_size) // 2, height - self.square_size)}
        self.eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'
        
        self.sound_played = {element: False for element in self.squares}
        self.finger_in_box = {element: False for element in self.squares}
        self.last_sound_time = {element: 0 for element in self.squares}
        
        self.grabbed_word = None
        self.gold_achieved = False
        self.mask_color = None
        
        self.reset_word_positions()
        
        # Use the Segoe UI Emoji font for rendering emojis
        self.emoji_font = ImageFont.truetype("seguiemj.ttf", 48)
        self.emoji_images = self.prepare_emoji_images()

    def reset_word_positions(self):
        self.word_positions = {element: (info['position'][0] + self.square_size // 2, info['position'][1] + self.square_size // 2) 
                               for element, info in self.squares.items()}
        self.gold_achieved = False
        self.grabbed_word = None
        self.mask_color = None

    def is_hand_closed(self, hand_landmarks):
        thumb_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.THUMB_TIP]
        index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
        return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

    def is_point_in_box(self, point, box_position, box_size):
        return (box_position[0] < point[0] < box_position[0] + box_size and
                box_position[1] < point[1] < box_position[1] + box_size)

    def prepare_emoji_images(self):
        emoji_images = {}
        for element in self.squares.keys():
            size = self.square_size
            emoji_images[element] = Image.new('RGBA', (size, size), (255, 255, 255, 0))
            d = ImageDraw.Draw(emoji_images[element])
            text_bbox = d.textbbox((0, 0), element, font=self.emoji_font)
            text_x = (size - (text_bbox[2] - text_bbox[0])) // 2
            text_y = (size - (text_bbox[3] - text_bbox[1])) // 2
            d.text((text_x, text_y), element, font=self.emoji_font, fill=(255, 255, 255, 255))
        return emoji_images

    def play_sound(self, sound_file):
        def play():
            try:
                winsound.PlaySound(sound_file, winsound.SND_FILENAME)
                print(f"Played sound: {sound_file}")
            except Exception as e:
                print(f"Error playing sound {sound_file}: {e}")

        threading.Thread(target=play, daemon=True).start()

    def combine_colors(self, colors):
        """Combine multiple RGB colors safely."""
        combined = np.clip(np.mean(colors, axis=0), 0, 255).astype(int)
        return tuple(combined)

    def validate_color(self, color):
        """Ensure the color is a valid RGB tuple."""
        if isinstance(color, tuple) and len(color) == 3:
            return tuple(int(c) for c in color)
        else:
            return (255, 255, 255)  # Fallback to white if invalid

    def process_frame(self, frame):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)

        overlay = image_rgb.copy()
        for element, info in self.squares.items():
            color = self.validate_color(info['color'])
            cv2.rectangle(overlay, info['position'], 
                          (info['position'][0] + self.square_size, info['position'][1] + self.square_size), 
                          color, -1)
        blended_image = cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0)

        color = self.validate_color(self.gold_box['color'])
        cv2.rectangle(blended_image, self.gold_box['position'], 
                      (self.gold_box['position'][0] + self.square_size, self.gold_box['position'][1] + self.square_size), 
                      color, -1)
        blended_image = cv2.addWeighted(blended_image, 0.25, image_rgb, 0.75, 0)

        pil_image = Image.fromarray(blended_image)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    blended_image, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                    self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
                )
                
                index_finger_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * self.width), int(index_finger_tip.y * self.height)

                hand_closed = self.is_hand_closed(hand_landmarks)

                if self.grabbed_word:
                    if hand_closed:
                        self.word_positions[self.grabbed_word] = (x, y)
                    else:
                        if self.is_point_in_box((x, y), self.gold_box['position'], self.square_size):
                            self.mask_color = self.squares[self.grabbed_word]['color']
                            self.gold_box['color'] = self.mask_color
                        self.grabbed_word = None
                else:
                    for element, info in self.squares.items():
                        if self.is_point_in_box((x, y), info['position'], self.square_size):
                            current_time = time.time()
                            if not self.finger_in_box[element]:
                                print(f"Touch detected in {element} square")
                                self.finger_in_box[element] = True
                                if current_time - self.last_sound_time[element] > 1:  # 1 second cooldown
                                    print(f"Attempting to play sound for {element}")
                                    self.play_sound(info['sound'])
                                    self.last_sound_time[element] = current_time
                            if hand_closed and not self.grabbed_word:
                                self.grabbed_word = element
                        else:
                            self.finger_in_box[element] = False

        # Check if multiple elements are in the gold box
        colors_in_gold = [self.squares[element]['color'] for element, position in self.word_positions.items() if self.is_point_in_box(position, self.gold_box['position'], self.square_size)]
        if len(colors_in_gold) > 1:
            combined_color = self.combine_colors(colors_in_gold)
            self.mask_color = self.validate_color(combined_color)
            self.gold_box['color'] = self.mask_color
            print(f"Combined color applied: {self.mask_color}")
        elif len(colors_in_gold) == 1:
            self.mask_color = self.validate_color(colors_in_gold[0])
            self.gold_box['color'] = self.mask_color

        for element, position in self.word_positions.items():
            pil_image.paste(self.emoji_images[element], (position[0] - self.square_size // 2, position[1] - self.square_size // 2), self.emoji_images[element])

        return np.array(pil_image)

class CameraManager:
    def __init__(self, cap):
        self.cap = cap
        self.is_game_view = True
        self.mp_selfie_segmentation = mp.solutions.selfie_segmentation
        self.selfie_segmentation = self.mp_selfie_segmentation.SelfieSegmentation(model_selection=0)

    def switch_view(self):
        self.is_game_view = not self.is_game_view
        message = 'Game View' if self.is_game_view else 'Mask View'
        print(f"Switched to {message}")

    def apply_mask(self, frame, mask_color, opacity):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.selfie_segmentation.process(image_rgb)
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        if mask_color is not None:
            overlay = np.full(image_rgb.shape, mask_color, dtype=np.uint8)
            blended_image = cv2.addWeighted(image_rgb, 1 - opacity, overlay, opacity, 0)
            image_rgb = np.where(condition, blended_image, image_rgb)
        return image_rgb

class MainProgram:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        ret, frame = self.cap.read()
        self.height, self.width = frame.shape[:2]
        self.game = ElementGame(self.cap, self.width, self.height)
        self.camera_manager = CameraManager(self.cap)

    def run(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to capture frame")
                break

            frame = cv2.flip(frame, 1)

            if self.camera_manager.is_game_view:
                image_rgb = self.game.process_frame(frame)
                image_rgb = self.camera_manager.apply_mask(image_rgb, self.game.mask_color, opacity=0.25)
            else:
                image_rgb = self.camera_manager.apply_mask(frame, self.game.mask_color, opacity=0.45)

            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            cv2.imshow('Camera View', image_bgr)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                self.game.reset_word_positions()
                print("Game reset!")
            elif key == ord('s'):
                self.camera_manager.switch_view()

        self.cap.release()
        cv2.destroyAllWindows()
        winsound.PlaySound(None, winsound.SND_PURGE)

if __name__ == "__main__":
    program = MainProgram()
    program.run()


Touch detected in 🌊 square
Attempting to play sound for 🌊
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav


//3RD ChatGPT iteration\\

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading
import os
import time
from PIL import Image, ImageDraw, ImageFont

class ElementGame:
    def __init__(self, cap, width, height):
        self.cap = cap
        self.width = width
        self.height = height
        self.square_size = min(width, height) // 4
        
        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
        
        # Initialize MediaPipe Selfie Segmentation
        self.mp_selfie_segmentation = mp.solutions.selfie_segmentation
        self.selfie_segmentation = self.mp_selfie_segmentation.SelfieSegmentation(model_selection=0)
        
        # Initialize drawing utilities
        self.mp_drawing = mp.solutions.drawing_utils
        
        self.squares = {
            '🔥': {'color': (255, 0, 0), 'position': (0, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav'},
            '💨': {'color': (173, 216, 230), 'position': (width - self.square_size, 0), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/air.wav'},
            '🌊': {'color': (0, 0, 139), 'position': (0, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/water.wav'},
            '🌱': {'color': (139, 69, 19), 'position': (width - self.square_size, height - self.square_size), 'sound': 'C:/Users/dsrus/Desktop/Workspace/MTLiens/earth.wav'}
        }
        
        self.gold_box = {'color': (0, 215, 255), 'position': ((width - self.square_size) // 2, height - self.square_size)}
        self.eureka_sound = 'C:/Users/dsrus/Desktop/Workspace/MTLiens/Eureka.wav'
        
        self.sound_played = {element: False for element in self.squares}
        self.finger_in_box = {element: False for element in self.squares}
        self.last_sound_time = {element: 0 for element in self.squares}
        
        self.grabbed_word = None
        self.gold_achieved = False
        self.mask_color = None
        
        self.reset_word_positions()
        
        self.emoji_font = ImageFont.truetype("seguiemj.ttf", 48)

    def reset_word_positions(self):
        self.word_positions = {element: (info['position'][0] + self.square_size // 2, info['position'][1] + self.square_size // 2) 
                               for element, info in self.squares.items()}
        self.gold_achieved = False
        self.grabbed_word = None
        self.mask_color = None

    def is_hand_closed(self, hand_landmarks):
        thumb_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.THUMB_TIP]
        index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
        return (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2 < 0.01

    def is_point_in_box(self, point, box_position, box_size):
        return (box_position[0] < point[0] < box_position[0] + box_size and
                box_position[1] < point[1] < box_position[1] + box_size)

    def play_sound(self, sound_file):
        def play():
            try:
                if not os.path.exists(sound_file):
                    print(f"Error: Sound file not found: {sound_file}")
                    return
                winsound.PlaySound(sound_file, winsound.SND_FILENAME)
                print(f"Played sound: {sound_file}")
            except Exception as e:
                print(f"Error playing sound {sound_file}: {e}")

        threading.Thread(target=play, daemon=True).start()

    def combine_colors(self, colors):
        """Combine multiple RGB colors safely."""
        combined = np.clip(np.mean(colors, axis=0), 0, 255).astype(int)
        return tuple(combined)

    def validate_color(self, color):
        """Ensure the color is a valid RGB tuple."""
        if isinstance(color, tuple) and len(color) == 3:
            return tuple(int(c) for c in color)
        else:
            return (255, 255, 255)  # Fallback to white if invalid

    def apply_mask(self, image_rgb):
        results = self.selfie_segmentation.process(image_rgb)
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        if self.mask_color is not None:
            overlay = np.zeros(image_rgb.shape, dtype=np.uint8)
            overlay[:] = self.mask_color
            overlay = cv2.addWeighted(image_rgb, 0.8, overlay, 0.2, 0)
            image_rgb = np.where(condition, overlay, image_rgb)
        return image_rgb

    def process_frame(self, frame):
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)

        # Apply mask first
        image_rgb = self.apply_mask(image_rgb)

        overlay = image_rgb.copy()
        for element, info in self.squares.items():
            cv2.rectangle(overlay, info['position'], 
                          (info['position'][0] + self.square_size, info['position'][1] + self.square_size), 
                          info['color'], -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        cv2.rectangle(overlay, self.gold_box['position'], 
                      (self.gold_box['position'][0] + self.square_size, self.gold_box['position'][1] + self.square_size), 
                      self.gold_box['color'] if self.mask_color is None else self.mask_color, -1)
        cv2.addWeighted(overlay, 0.25, image_rgb, 0.75, 0, image_rgb)

        pil_image = Image.fromarray(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    image_rgb, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                    self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
                )
                
                index_finger_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger_tip.x * self.width), int(index_finger_tip.y * self.height)

                hand_closed = self.is_hand_closed(hand_landmarks)

                if self.grabbed_word:
                    if hand_closed:
                        self.word_positions[self.grabbed_word] = (x, y)
                    else:
                        if self.is_point_in_box((x, y), self.gold_box['position'], self.square_size):
                            self.mask_color = self.squares[self.grabbed_word]['color']
                            self.gold_box['color'] = self.mask_color
                        self.grabbed_word = None
                else:
                    for element, info in self.squares.items():
                        if self.is_point_in_box((x, y), info['position'], self.square_size):
                            current_time = time.time()
                            if not self.finger_in_box[element]:
                                print(f"Touch detected in {element} square")
                                self.finger_in_box[element] = True
                                if current_time - self.last_sound_time[element] > 1:  # 1 second cooldown
                                    print(f"Attempting to play sound for {element}")
                                    self.play_sound(info['sound'])
                                    self.last_sound_time[element] = current_time
                            if hand_closed and not self.grabbed_word:
                                self.grabbed_word = element
                        else:
                            self.finger_in_box[element] = False

        # Check if multiple elements are in the gold box
        colors_in_gold = [self.squares[element]['color'] for element, position in self.word_positions.items() if self.is_point_in_box(position, self.gold_box['position'], self.square_size)]
        if len(colors_in_gold) > 1:
            combined_color = self.combine_colors(colors_in_gold)
            self.mask_color = self.validate_color(combined_color)
            self.gold_box['color'] = self.mask_color
            print(f"Combined color applied: {self.mask_color}")
        elif len(colors_in_gold) == 1:
            self.mask_color = self.validate_color(colors_in_gold[0])
            self.gold_box['color'] = self.mask_color

        for element, position in self.word_positions.items():
            draw = ImageDraw.Draw(pil_image)
            draw.text((position[0], position[1]), element, font=self.emoji_font, fill=(255, 255, 255), anchor="mm")

        return np.array(pil_image)

class CameraManager:
    def __init__(self, cap):
        self.cap = cap
        self.is_game_view = True

    def switch_view(self):
        self.is_game_view = not self.is_game_view
        print(f"Switched to {'game' if self.is_game_view else 'mask'} view")

class MainProgram:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        ret, frame = self.cap.read()
        self.height, self.width = frame.shape[:2]
        self.game = ElementGame(self.cap, self.width, self.height)
        self.camera_manager = CameraManager(self.cap)

    def run(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to capture frame")
                break

            frame = cv2.flip(frame, 1)

            if self.camera_manager.is_game_view:
                image_rgb = self.game.process_frame(frame)
            else:
                image_rgb = self.game.apply_mask(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            cv2.imshow('Camera View', image_bgr)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                self.game.reset_word_positions()
                print("Game reset!")
            elif key == ord('s'):
                self.camera_manager.switch_view()

        self.cap.release()
        cv2.destroyAllWindows()
        winsound.PlaySound(None, winsound.SND_PURGE)

if __name__ == "__main__":
    program = MainProgram()
    program.run()


Touch detected in 🔥 square
Attempting to play sound for 🔥
Touch detected in 🔥 square
Switched to mask view
Played sound: C:/Users/dsrus/Desktop/Workspace/MTLiens/fire.wav
Switched to game view
Switched to mask view
Switched to game view
Touch detected in 🌊 square
Attempting to play sound for 🌊
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Combined color applied: (127, 0, 69)
Co